In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key=''

# anthropic API key
claude_api_key = ''

# together AI API key
together_api_key = ''


In [ ]:
# read the prompt from the prompt text file titled "chess_matchmaking.txt"
with open("../prompts/chess_matchmaking.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)
print(prompt)

In [3]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT 4o

In [8]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [ ]:
print(response)

In [ ]:
# check if there is code in the response
pattern = r'```python(.*?)```'
matches = re.findall(pattern, response, re.DOTALL)
if matches:
    code = matches[0].strip()
    print(code)
else:
    print("No code found in the response.")

In [19]:
import numpy as np
from scipy.stats import truncnorm

# Define parameters
elo_mean = 1200.0
elo_sd = 515.8299897407918
poisson_rate = 1.0
num_players = 1000
delta = 5.0
budget = 1000   # Adjust this as needed for testing different budgets
min_x, max_x = 0, 2400

# Truncated normal distribution for ratings
a, b = (0 - elo_mean) / elo_sd, (2400 - elo_mean) / elo_sd

def simulate_matching_process(x, num_players, lambda_rate, elo_mean, elo_sd, a, b):
    # Generate player ratings
    ratings = truncnorm.rvs(a, b, loc=elo_mean, scale=elo_sd, size=num_players)
    
    # Generate arrival times as a Poisson process
    arrival_times = np.cumsum(np.random.exponential(1 / lambda_rate, size=num_players))
    
    # Initialize waiting pool and performance metrics
    waiting_pool = []
    total_elo_diff = 0
    total_wait_time = 0
    num_matches = 0
    
    for i in range(num_players):
        current_player = (ratings[i], arrival_times[i])
        
        # Try to find a match from the waiting pool
        match_found = False
        for j, (waiting_rating, waiting_time) in enumerate(waiting_pool):
            if abs(waiting_rating - current_player[0]) <= x:
                # Match found
                total_elo_diff += abs(waiting_rating - current_player[0])
                total_wait_time += current_player[1] - waiting_time
                num_matches += 1
                
                # Remove the matched player from the pool
                waiting_pool.pop(j)
                match_found = True
                break
        
        # If no match was found, add player to the waiting pool
        if not match_found:
            waiting_pool.append(current_player)
    
    avg_diff = total_elo_diff / num_matches if num_matches > 0 else float('inf')
    avg_wait_time = total_wait_time / num_matches if num_matches > 0 else float('inf')
    
    return avg_diff, avg_wait_time

# Optimize x within budget
optimal_x = min_x
best_avg_diff = float('inf')

for x in np.linspace(min_x, max_x, num=budget):
    avg_diff, avg_wait_time = simulate_matching_process(x, num_players, poisson_rate, elo_mean, elo_sd, a, b)
    
    if avg_wait_time <= delta and avg_diff < best_avg_diff:
        best_avg_diff = avg_diff
        optimal_x = x

print(f"Recommended allowable_diff (x): {optimal_x}")
print(f"Resulting avg_diff: {best_avg_diff}")


# run the above thing 1000 times to get mean and std for the best_avg_diff
num_trials = 1000
avg_diffs = []
for _ in range(num_trials):
    avg_diff, _ = simulate_matching_process(optimal_x, num_players, poisson_rate, elo_mean, elo_sd, a, b)
    avg_diffs.append(avg_diff)

# print the mean and std dev for the avg_diff
mean_avg_diff = np.mean(avg_diffs)
std_avg_diff = np.std(avg_diffs)
best_avg_diff = mean_avg_diff
print(f"Mean avg_diff over {num_trials} trials: {mean_avg_diff:.4f}")
print(f"Standard deviation of avg_diff over {num_trials} trials: {std_avg_diff:.4f}")

Recommended allowable_diff (x): 252.25225225225225
Resulting avg_diff: 121.00389757416951
Mean avg_diff over 1000 trials: 121.9225
Standard deviation of avg_diff over 1000 trials: 3.3591


## o1

In [20]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [ ]:
print(response)

In [23]:
import math
import random
import statistics
import bisect

# --------------------------
# Helper: truncated normal sampler
# --------------------------
def truncated_normal(mean, sd, lower, upper):
    """
    Simple acceptance-rejection sampling from a Normal distribution
    truncated to [lower, upper].
    """
    while True:
        # sample from normal
        x = random.gauss(mean, sd)
        if lower <= x <= upper:
            return x

# --------------------------
# Main Simulation
# --------------------------
def run_simulation(num_players=1000,
                   lambda_rate=1.0,
                   elo_mean=1200.0,
                   elo_sd=1200.0/(math.sqrt(2)*0.5),  # i.e., 1/(erfcinv(1/50)) ~ 0.5
                   elo_lower=0.0,
                   elo_upper=2400.0,
                   allowable_diff=150.0,
                   seed=42):
    """
    Runs a Monte Carlo simulation for 'num_players' arrivals with the given
    Poisson rate, rating distribution, and matching threshold.

    Returns:
        avg_diff (float): average Elo difference in matched pairs
        avg_wait_time (float): average waiting time
    """
    random.seed(seed)

    # Generate arrival times from a Poisson process
    arrival_times = []
    current_time = 0.0
    for _ in range(num_players):
        # Add an exponential(1/lambda_rate) increment
        inter_arrival = random.expovariate(lambda_rate)
        current_time += inter_arrival
        arrival_times.append(current_time)

    # Generate player ratings (truncated normal)
    ratings = [
        truncated_normal(elo_mean, elo_sd, elo_lower, elo_upper)
        for _ in range(num_players)
    ]

    # Sort by arrival to ensure correct chronological order
    # They are already in chronological order, but let's store them as list of (time, rating)
    players = list(zip(arrival_times, ratings))

    # We'll keep track of waiting players as a list of (rating, arrival_time).
    # Additionally, to speed up searching for a match, we can keep it sorted by rating.
    # However, for simplicity, we can keep a normal list, or we can do a binary search approach.
    waiting_pool = []
    matched_diffs = []
    waiting_times = []

    for arrival_time, rating in players:
        # Find a match in the waiting pool
        # A naive approach: traverse the waiting list to find the first that fits rating difference <= x
        # In production, we might keep a sorted structure, but let's do brute force for clarity.

        match_index = None
        for i, (w_rating, w_arrival) in enumerate(waiting_pool):
            if abs(rating - w_rating) <= allowable_diff:
                match_index = i
                break

        if match_index is not None:
            # Match found
            w_rating, w_arrival = waiting_pool.pop(match_index)

            # Record waiting times
            wait_time_waiting_player = arrival_time - w_arrival
            waiting_times.append(wait_time_waiting_player)
            # The new arrival is matched right away
            waiting_times.append(0.0)

            # Record rating difference
            matched_diffs.append(abs(rating - w_rating))

        else:
            # No match, so add this new player to the pool
            waiting_pool.append((rating, arrival_time))

    # After all arrivals, some players remain unmatched;
    # in real systems, we only measure matched players' wait times.
    # We'll ignore unmatched players for this calculation.

    if matched_diffs:
        avg_diff = statistics.mean(matched_diffs)
    else:
        avg_diff = float('nan')

    if waiting_times:
        avg_wait_time = statistics.mean(waiting_times)
    else:
        avg_wait_time = 0.0

    return avg_diff, avg_wait_time

def find_best_x(budget=20,
                x_min=0,
                x_max=2400,
                delta=5.0,
                num_players=1000,
                lambda_rate=1.0,
                seed=42):
    """
    Search for the best allowable x within [x_min, x_max] to
    minimize rating difference subject to avg_wait_time <= delta.

    We do a simple uniform grid search with given 'budget' points.
    """
    best_x = None
    best_diff = float('inf')

    # Construct a grid of x values
    xs = [x_min + i*(x_max - x_min)/(budget - 1) for i in range(budget)]

    for x in xs:
        avg_diff, avg_wait = run_simulation(num_players=num_players,
                                            lambda_rate=lambda_rate,
                                            allowable_diff=x,
                                            seed=seed)
        if avg_wait <= delta and avg_diff < best_diff:
            best_diff = avg_diff
            best_x = x

    return best_x, best_diff

if __name__ == "__main__":
    # Example usage:
    x_candidate, diff_candidate = find_best_x(budget=1000,  # fewer points for quick demo
                                              x_min=0,
                                              x_max=300,   # searching up to 300
                                              delta=5.0,
                                              num_players=1000,
                                              lambda_rate=1.0,
                                              seed=42)
    # Compare to baseline x=150
    baseline_diff, baseline_wait = run_simulation(num_players=1000,
                                                 lambda_rate=1.0,
                                                 allowable_diff=150,
                                                 seed=42)
    print(f"Optimized x = {x_candidate:.1f}, avg_diff = {diff_candidate:.1f} (if feasible)")
    print(f"Baseline x=150 => avg_diff={baseline_diff:.1f}, avg_wait={baseline_wait:.2f}")

    # run the above thing 1000 times to get mean and std for the best_avg_diff
    num_trials = 1000
    avg_diffs = []
    for _ in range(num_trials):
        avg_diff, _ = run_simulation(num_players=1000,
                                     lambda_rate=1.0,
                                     allowable_diff=x_candidate,
                                     seed=_)
        avg_diffs.append(avg_diff)
    # print the mean and std dev for the avg_diff
    mean_avg_diff = statistics.mean(avg_diffs)
    std_avg_diff = statistics.stdev(avg_diffs) if len(avg_diffs) > 1 else 0.0
    print(f"Mean avg_diff over {num_trials} trials: {mean_avg_diff:.4f}")
    print(f"Standard deviation of avg_diff over {num_trials} trials: {std_avg_diff:.4f}")
    # Final report

Optimized x = 137.8, avg_diff = 65.4 (if feasible)
Baseline x=150 => avg_diff=72.9, avg_wait=4.54
Mean avg_diff over 1000 trials: 65.9541
Standard deviation of avg_diff over 1000 trials: 1.8059


## o3-mini

In [27]:
completion = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [28]:
print(response)

Below is one way to “solve” the problem. In our write‐up we explain our reasoning and then provide a single self‐contained Python code block that:

• Generates num_players ratings from a truncated normal (by rejection‐sampling) with the given mean and sd on [0,2400].

• Generates arrival times from a Poisson process (by taking exponential inter–arrival times with rate λ).

• Implements a “matching” procedure. When a new arrival comes we look in the waiting pool for a player whose rating is within ±x (the candidate “allowable_diff”) of the new player. (We choose, say, the earliest waiting player among those that qualify.) If one is found, we record (a) the absolute difference of their ratings and (b) the waiting time (new time minus waiting player’s arrival time). If not, the new player simply is added to the waiting pool.

• Repeats the above for the simulation and then computes the overall average Elo difference and average waiting time.

Because in theory one expects a trade‐off – a 

In [31]:
#!/usr/bin/env python3
import numpy as np
import random
import math
import time

# For reproducibility
np.random.seed(42)
random.seed(42)

def truncated_normal_samples(n, mean, sd, low, high):
    """Generate n samples from a truncated normal distribution via rejection sampling."""
    samples = []
    while len(samples) < n:
        # Oversample by factor 2:
        candidate = np.random.normal(mean, sd, size=n)
        candidate = candidate[(candidate >= low) & (candidate <= high)]
        samples.extend(candidate.tolist())
    return np.array(samples[:n])
    
def simulate_matching(allowable_diff, poisson_rate=1.0, num_players=1000,
                      elo_mean=1200.0, elo_sd=None, delta=5.0):
    """
    Simulate the matching process.
    
    Inputs:
      allowable_diff     : maximum allowed Elo difference for matching (x)
      poisson_rate       : arrival rate of players
      num_players        : total players to simulate (each arrival gets a rating)
      elo_mean           : mean rating
      elo_sd             : rating standard deviation (if None, compute from given formula)
      delta              : upper bound on average waiting time constraint
      
    Outputs:
      avg_diff           : average Elo difference among matched pairs (for matches formed)
      avg_wait_time      : average waiting time for matching (waiting player reported waiting time)
      n_matched          : number of matches completed
    """
    # If elo_sd not provided, use the given formula:
    if elo_sd is None:
        # The given formula: 1200 / (sqrt(2)*erfcinv(1/50)).
        # We compute erfcinv using math.erfc and a numerical inversion.
        # For our simulation we hard–code this value.
        # One may note that in our truncated distribution 0 and 2400 are about 1st and 99th percentiles.
        # A typical value turns out to be around:
        elo_sd = 1200 / (math.sqrt(2) * 2.376)  # approximate value since erfcinv(1/50) ~ 2.376
        # This yields elo_sd ~ 318. Now use that.
    
    # Generate players’ ratings (truncated to [0,2400])
    ratings = truncated_normal_samples(num_players, elo_mean, elo_sd, 0, 2400)
    
    # Generate arrival times from a Poisson process:
    inter_arrivals = np.random.exponential(1.0/poisson_rate, size=num_players)
    arrival_times = np.cumsum(inter_arrivals)
    
    # waiting pool: list of dicts, each with keys: 'arrival_time', 'rating'
    waiting_pool = []
    paired_diffs = []   # store absolute rating differences for matches
    wait_times = []     # store waiting time for the waiting player in a match
    
    # Simulation: process each arriving player in order
    for idx in range(num_players):
        current_player = {'arrival_time': arrival_times[idx], 'rating': ratings[idx]}
        # Check if there is any waiting player in the pool whose rating is close enough.
        # We choose the first waiting player (i.e., earliest arrival) that qualifies.
        match_found = False
        for j, waiting_player in enumerate(waiting_pool):
            if abs(waiting_player['rating'] - current_player['rating']) <= allowable_diff:
                # We found a match:
                paired_diffs.append(abs(waiting_player['rating'] - current_player['rating']))
                wait_times.append(current_player['arrival_time'] - waiting_player['arrival_time'])
                # Remove the waiting player from the pool
                waiting_pool.pop(j)
                match_found = True
                break
        if not match_found:
            # In case no waiting partner exists, just add the current player to the waiting pool.
            waiting_pool.append(current_player)
    
    # For unmatched players, we do not count waiting time or pairing difference.
    if paired_diffs:
        avg_diff = np.mean(paired_diffs)
        avg_wait_time = np.mean(wait_times)
    else:
        avg_diff = None
        avg_wait_time = None
        
    return avg_diff, avg_wait_time, len(paired_diffs)
    
def search_optimal_threshold(budget=1000, 
                             poisson_rate=1.0, num_players=1000, delta=5.0,
                             elo_mean=1200.0):
    """
    Search over candidate allowable_diff (x) values on [0, 2400] to find the smallest
    x that yields an average wait time no greater than delta.
    
    We use a grid search with [budget] candidate values.
    For each candidate we run one simulation.
    In practice one may average over several runs if desired.
    
    Returns:
      best_x (the recommended allowable_diff),
      a results list with tuples (x, avg_diff, avg_wait_time, n_matched)
    """
    # We assume x must be between a minimal value and 2400.
    # (We use a lower bound of 0 or maybe a small value, say 10, to avoid pathological no-match behavior)
    candidate_xs = np.linspace(10, 2400, num=budget)
    results = []
    solution_candidates = []
    
    for x in candidate_xs:
        avg_diff, avg_wait_time, n_matched = simulate_matching(x,
                                                              poisson_rate=poisson_rate,
                                                              num_players=num_players,
                                                              elo_mean=elo_mean,
                                                              delta=delta)
        results.append((x, avg_diff, avg_wait_time, n_matched))
        # We will only consider candidate thresholds for which the average waiting time is within delta.
        if avg_wait_time is not None and avg_wait_time <= delta:
            solution_candidates.append((x, avg_diff, avg_wait_time, n_matched))
    
    # Our objective is to minimize average Elo difference subject to waiting time <= delta.
    # Under many circumstances one finds that as x increases, waiting time decreases,
    # so the “optimal” strategy is to choose the smallest x that yields the constraint.
    if solution_candidates:
        # sort by x; choose the candidate with the smallest x that meets the constraint.
        solution_candidates.sort(key=lambda tup: tup[0])
        best_candidate = solution_candidates[0]
    else:
        best_candidate = None
        
    return best_candidate, results

# Run a test simulation search with a given budget, arrival rate, etc.
if __name__ == '__main__':
    start_time = time.time()
    # Parameters (these can be changed to test different cases):
    poisson_rate = 1.0
    num_players = 1000
    delta = 5.0  # waiting time constraint (upper bound)
    budget = 1000   # number of candidate x values to test
    elo_mean = 1200.0

    best_candidate, results = search_optimal_threshold(budget=budget, poisson_rate=poisson_rate,
                                                       num_players=num_players,
                                                       delta=delta, elo_mean=elo_mean)
    
    if best_candidate is not None:
        best_x, best_avg_diff, best_avg_wait, matched = best_candidate
        print("Recommended allowable_diff value: {:.2f}".format(best_x))
        print("Resulting average Elo difference: {:.2f}".format(best_avg_diff))
        print("Resulting average waiting time: {:.2f}".format(best_avg_wait))
        print("Number of matches made: {}".format(matched))
    else:
        print("No candidate threshold met the waiting time constraint of {:.2f}".format(delta))
    
    elapsed = time.time() - start_time
    print("Search completed in {:.2f} seconds.".format(elapsed))
    
    # For reference, you may compare with a baseline of x=150.
    baseline_x = 150.0
    base_diff, base_wait, base_matches = simulate_matching(baseline_x, poisson_rate=poisson_rate,
                                                            num_players=num_players,
                                                            elo_mean=elo_mean,
                                                            delta=delta)
    if base_diff is not None:
        print("\nBaseline (x=150) results:")
        print("Average Elo difference: {:.2f}".format(base_diff))
        print("Average waiting time: {:.2f}".format(base_wait))
        print("Number of matches made: {}".format(base_matches))
    else:
        print("Baseline simulation produced no matches.")

    # run the above thing 1000 times to get mean and std for the best_avg_diff
    num_trials = 1000

    avg_diffs = []
    for _ in range(num_trials):
        avg_diff, _, _ = simulate_matching(best_x,
                                           poisson_rate=poisson_rate,
                                           num_players=num_players,
                                           elo_mean=elo_mean,
                                           delta=delta)
        if avg_diff is not None:
            avg_diffs.append(avg_diff)
    
    if avg_diffs:
        mean_avg_diff = np.mean(avg_diffs)
        std_avg_diff = np.std(avg_diffs)
        print(f"\nMean average Elo difference over {num_trials} trials: {mean_avg_diff:.4f}")
        print(f"Standard deviation of average Elo difference: {std_avg_diff:.4f}")

Recommended allowable_diff value: 208.57
Resulting average Elo difference: 97.78
Resulting average waiting time: 4.87
Number of matches made: 498
Search completed in 1.93 seconds.

Baseline (x=150) results:
Average Elo difference: 69.92
Average waiting time: 8.63
Number of matches made: 497

Mean average Elo difference over 1000 trials: 100.9898
Standard deviation of average Elo difference: 2.5870


## Claude Sonnet

In [32]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll help solve this matching optimization problem through a combination of analytical insights and simulation. Let me break down the approach:

1) Analytical Insights:
----------------------
The problem involves a queueing system with a matching constraint. While an exact analytical solution is complex due to the state-dependent matching rules, we can make some observations:

- For any given threshold x, the probability of finding a match is related to the CDF of the normal distribution within ±x of a player's rating
- The waiting time will increase as x decreases (fewer potential matches)
- There's a clear trade-off between matching quality (low Elo difference) and waiting time

2) Proposed Solution Approach:
-----------------------------
I'll implement a binary search over potential x values, using simulation to evaluate each candidate. Here's the code:

```python
import numpy as np
from scipy.stats import truncnorm
import heapq
from collections import defaultdict

class ChessMatchi

In [33]:
import numpy as np
from scipy.stats import truncnorm
import heapq
from collections import defaultdict

class ChessMatchingSimulator:
    def __init__(self, elo_mean=1200.0, arrival_rate=1.0, num_players=1000):
        self.elo_mean = elo_mean
        self.elo_sd = 1200 / (np.sqrt(2) * -np.log(1/100))  # Approximating erfcinv
        self.arrival_rate = arrival_rate
        self.num_players = num_players
        
    def generate_ratings(self):
        a, b = (0 - self.elo_mean) / self.elo_sd, (2400 - self.elo_mean) / self.elo_sd
        ratings = truncnorm.rvs(a, b, loc=self.elo_mean, scale=self.elo_sd, size=self.num_players)
        return ratings
        
    def simulate_matching(self, allowable_diff):
        # Generate players and arrival times
        ratings = self.generate_ratings()
        arrivals = np.random.exponential(1/self.arrival_rate, self.num_players)
        arrival_times = np.cumsum(arrivals)
        
        waiting_players = []  # (rating, arrival_time, player_id)
        matches = []  # (player1_id, player2_id, wait_time, elo_diff)
        
        for i, (rating, arrival) in enumerate(zip(ratings, arrival_times)):
            # Try to match with waiting player
            best_match = None
            best_diff = float('inf')
            best_idx = -1
            
            for j, (w_rating, w_arrival, w_id) in enumerate(waiting_players):
                diff = abs(rating - w_rating)
                if diff <= allowable_diff and diff < best_diff:
                    best_match = (w_id, w_arrival)
                    best_diff = diff
                    best_idx = j
            
            if best_match is not None:
                # Make the match
                w_id, w_arrival = best_match
                wait_time = max(arrival - w_arrival, 0)
                matches.append((w_id, i, wait_time, best_diff))
                waiting_players.pop(best_idx)
            else:
                # Add to waiting pool
                waiting_players.append((rating, arrival, i))
                
        # Calculate metrics
        if not matches:
            return float('inf'), float('inf')
        
        avg_diff = np.mean([m[3] for m in matches])
        avg_wait = np.mean([m[2] for m in matches])
        
        return avg_diff, avg_wait

    def binary_search_threshold(self, target_wait_time, tolerance=0.1):
        left, right = 0, 2400
        best_x = None
        best_metrics = None
        
        while (right - left) > tolerance:
            x = (left + right) / 2
            avg_diff, avg_wait = self.simulate_matching(x)
            
            if avg_wait <= target_wait_time:
                # This is feasible, try to reduce x
                if best_x is None or avg_diff < best_metrics[0]:
                    best_x = x
                    best_metrics = (avg_diff, avg_wait)
                right = x
            else:
                # Need to increase x
                left = x
                
        return best_x, best_metrics

def optimize_matching(budget=1000):
    simulator = ChessMatchingSimulator()
    
    # Use binary search within budget
    optimal_x, (final_diff, final_wait) = simulator.binary_search_threshold(
        target_wait_time=5.0
    )
    
    return {
        'recommended_threshold': optimal_x,
        'avg_elo_difference': final_diff,
        'avg_wait_time': final_wait
    }

# Example usage
if __name__ == "__main__":
    result = optimize_matching(budget=1000)
    print(f"Recommended allowable_diff (x): {result['recommended_threshold']:.2f}")
    print(f"Resulting average Elo difference: {result['avg_elo_difference']:.2f}")
    print(f"Resulting average waiting time: {result['avg_wait_time']:.2f}")

    # run the code 1000 times to get mean and std for the best_avg_diff
    num_trials = 1000
    avg_diffs = []
    for _ in range(num_trials):
        simulator = ChessMatchingSimulator()
        avg_diff, _ = simulator.simulate_matching(result['recommended_threshold'])
        avg_diffs.append(avg_diff)
    mean_avg_diff = np.mean(avg_diffs)
    std_avg_diff = np.std(avg_diffs) if len(avg_diffs) > 1 else 0.0
    print(f"Mean average Elo difference over {num_trials} trials: {mean_avg_diff:.4f}")
    print(f"Standard deviation of average Elo difference: {std_avg_diff:.4f}")
    

Recommended allowable_diff (x): 107.81
Resulting average Elo difference: 49.97
Resulting average waiting time: 4.90
Mean average Elo difference over 1000 trials: 50.1031
Standard deviation of average Elo difference: 1.3399


## DeepSeek R1

In [34]:
import numpy as np
import bisect

def generate_elos(num_players):
    elo_mean = 1200.0
    erfcinv_0_02 = 1.6448536269514722  # Precomputed using scipy's erfcinv(0.02)
    elo_sd = 1200 / (np.sqrt(2) * erfcinv_0_02)
    elos = np.random.normal(elo_mean, elo_sd, num_players)
    elos = np.clip(elos, 0, 2400)
    return elos

def generate_arrival_times(num_players, poisson_rate):
    intervals = np.random.exponential(scale=1/poisson_rate, size=num_players)
    arrival_times = np.cumsum(intervals)
    return arrival_times

def simulate(elos, arrival_times, x):
    waiting_elos = []  # List of tuples (elo, arrival_time), sorted by elo
    matched_diffs = []
    matched_waits = []
    
    for i in range(len(elos)):
        current_elo = elos[i]
        current_time = arrival_times[i]
        
        lower = current_elo - x
        upper = current_elo + x
        
        left = bisect.bisect_left(waiting_elos, (lower, -np.inf))
        right = bisect.bisect_right(waiting_elos, (upper, np.inf))
        candidates = waiting_elos[left:right]
        
        if not candidates:
            bisect_idx = bisect.bisect_left(waiting_elos, (current_elo, current_time))
            waiting_elos.insert(bisect_idx, (current_elo, current_time))
            continue
        
        min_diff = np.inf
        best_idx = -1
        best_arrival = 0.0
        for idx in range(len(candidates)):
            elo, arrival = candidates[idx]
            diff = abs(current_elo - elo)
            if diff < min_diff:
                min_diff = diff
                best_arrival = arrival
                best_idx = left + idx
        
        if min_diff <= x:
            matched_diffs.append(min_diff)
            wait_existing = current_time - best_arrival
            matched_waits.append(wait_existing)
            matched_waits.append(0.0)
            del waiting_elos[best_idx]
        else:
            bisect_idx = bisect.bisect_left(waiting_elos, (current_elo, current_time))
            waiting_elos.insert(bisect_idx, (current_elo, current_time))
    
    if not matched_diffs:
        return 0.0, 0.0
    avg_diff = sum(matched_diffs) / len(matched_diffs)
    avg_wait = sum(matched_waits) / len(matched_waits)
    return avg_diff, avg_wait

def find_optimal_x(budget, delta=5.0, num_players=1000, poisson_rate=1.0):
    best_x = None
    best_avg_diff = np.inf
    best_avg_wait = np.inf
    x_candidates = np.linspace(0, 2400, budget)
    
    for x in x_candidates:
        elos = generate_elos(num_players)
        arrival_times = generate_arrival_times(num_players, poisson_rate)
        avg_diff, avg_wait = simulate(elos, arrival_times, x)
        
        if avg_wait <= delta and avg_diff < best_avg_diff:
            best_x = x
            best_avg_diff = avg_diff
            best_avg_wait = avg_wait
    
    if best_x is None:
        best_x = 150.0
        elos = generate_elos(num_players)
        arrival_times = generate_arrival_times(num_players, poisson_rate)
        best_avg_diff, best_avg_wait = simulate(elos, arrival_times, best_x)
    
    return best_x, best_avg_diff, best_avg_wait

if __name__ == "__main__":
    budget = 1000
    optimal_x, avg_diff, avg_wait = find_optimal_x(budget)
    print(f"Recommended allowable_diff: {optimal_x:.2f}")
    print(f"Average Elo difference: {avg_diff:.2f}")
    print(f"Average waiting time: {avg_wait:.2f}")

    # run the above thing 1000 times to get mean and std for the best_avg_diff
    num_trials = 1000
    avg_diffs = []
    for _ in range(num_trials):
        elos = generate_elos(1000)
        arrival_times = generate_arrival_times(1000, 1.0)
        avg_diff, _ = simulate(elos, arrival_times, optimal_x)
        avg_diffs.append(avg_diff)
    mean_avg_diff = np.mean(avg_diffs)
    std_avg_diff = np.std(avg_diffs) if len(avg_diffs) > 1 else 0.0
    print(f"Mean average Elo difference over {num_trials} trials: {mean_avg_diff:.4f}")
    print(f"Standard deviation of average Elo difference: {std_avg_diff:.4f}")

Recommended allowable_diff: 134.53
Average Elo difference: 61.49
Average waiting time: 4.80
Mean average Elo difference over 1000 trials: 61.8869
Standard deviation of average Elo difference: 1.7036
